# Stratal EDL Learner

In [0]:
import numpy as np
import re
import operator
from random import choice
from sys import exit
from matplotlib.pyplot import plot, ylabel, show, xlabel, ylim, xlim, legend, title
from scipy.stats import sem
from google.colab import drive

In [0]:
###USER SETTINGS###
EPOCHS = 100
SAMPLES = 50
STRAT_NUM = 2 #This has to be 2 right now
LEARNING_RATE = .05
LANGUAGE = "esi_CB"
WD = "Opacity"
REPS = 10
TEST_DATUM = "ise"
#TEST_DATUM_OUTPUTS = {"f":"esi", "o":"eSe", "t":"ese", "jp":"eSi"} #F/CF   {faithful, opaque, transparent, just palatalization}
TEST_DATUM_OUTPUTS = {"f":"ise", "o":"isi", "t":"iSi", "jp":"iSe"} #B/CB    {faithful, opaque, transparent, just palatalization}

In [0]:
#Mount the user's google drive:
drive.mount('/content/gdrive', force_remount=True)

## Process Input

In [0]:
###PROCESS INPUT###
#CON:
CON_file = open("/content/gdrive/My Drive/"+WD+"/Input Files/"+LANGUAGE+"_CON.txt", "r")
marked_con = {}
faithful_con = {}
mark = False
for line in CON_file.readlines():
    if "markedness" in line:
        mark = True
        continue
    elif "faithfulness" in line:
        mark = False
        continue
        
    if mark:
        name, regex = line.rstrip().split("\t")
        marked_con[name] = regex
    else:
        name, this_ur, this_sr = line.rstrip().split("\t")
        faithful_con[name] = [this_ur.split(","), this_sr.split(",")]
CON = list(marked_con.keys()) + list(faithful_con.keys())
CON_file.close()

#GEN:
GEN_file = open("/content/gdrive/My Drive/"+WD+"/Input Files/"+LANGUAGE+"_GEN.txt", "r")
ur2cands = {}
for line in GEN_file.readlines():
    this_in, these_outs = line.rstrip().split("\t")
    cands = these_outs.split(",")
    ur2cands[this_in] = cands
GEN_file.close()

#Training Data
TD_file = open("/content/gdrive/My Drive/"+WD+"/Input Files/"+LANGUAGE+"_Dist.txt", "r")
TD_input = []
TD_output = []
for line in TD_file.readlines():
    data_line = re.search('"x"\t"(.+)"\t\t([0-9]+)\t[0-9]+\t(.+)', line)
    if data_line:
        this_sr = data_line.group(1)
        this_freq = int(data_line.group(2))
        this_ur = data_line.group(3)
        
        TD_input += [this_ur] * this_freq
        TD_output += [this_sr] * this_freq           
TD_file.close()



## Build Model

In [0]:
#Violation profiles:
get_viol = {}
for this_ur in ur2cands.keys():
    for cand in ur2cands[this_ur]:
        #Markedness
        surface_cand = re.sub("_", "", cand)
        for con in marked_con.keys():
            try:
                get_viol[(this_ur,cand)][con] = len(re.findall(marked_con[con], surface_cand))
            except:
                get_viol[(this_ur,cand)] = {}
                get_viol[(this_ur,cand)][con] = len(re.findall(marked_con[con], surface_cand))
        
        #Faithfulness
        for con in faithful_con.keys():
            get_viol[(this_ur,cand)][con] = 0
            for faith_index, faith_input_seg in enumerate(faithful_con[con][0]):
                faith_output_seg = faithful_con[con][1][faith_index]
                for form_index in range(len(this_ur)):
                    if (this_ur[form_index] == faith_input_seg) and (cand[form_index]==faith_output_seg):
                        get_viol[(this_ur,cand)][con] += 1
                        
#Rankings we're keeping track of:
    rankings = []
    for c1 in CON:
        for c2 in CON:
            if c1 == c2:
                continue
            rankings.append(">>".join(sorted([c1, c2])))
    rankings = list(set(rankings)) #Get rid of duplicate rankings
    strat_size = len(rankings) #num of cons in each stratum
    rankings *= STRAT_NUM #Copy the constraint set for multiple strata

###CUSTOM FUNCTIONS###
#Function for finding optimal candidate:
def getWinner (inp, cons, cands):
    new_cands = cands[:]
    for con in cons:
        these_viols = []
        violNum2cands = {}
        for cand in cands:
            this_viol = get_viol[(inp, cand)][con]
            these_viols.append(this_viol)
            try:
                violNum2cands[this_viol].append(cand)
            except:
                violNum2cands[this_viol] = [cand]
        new_cands = violNum2cands[min(these_viols)]
        if len(new_cands) == 1:
            return new_cands[0]
        else:
            cands = new_cands[:]
    return choice(cands)

#Function that converts stratum's rankings to ordered constraints:
def orderConstraints (stratum_names, stratum_ranks):
    dom_counts = {c:0 for c in CON}
    for rank_j, rank_name in enumerate(stratum_names):
        if stratum_ranks[rank_j]:
            dom_counts[rank_name.split(">>")[0]] += 1
        else:
            dom_counts[rank_name.split(">>")[1]] += 1  
                    
    ranked_cons = []
    for constraint in sorted(dom_counts.items(), key=operator.itemgetter(1), reverse=True):
        ranked_cons.append(constraint[0])
    
    return ranked_cons

#Function for getting accuracy of a given grammar:
def getAccuracy (grammar):
    samp_accs = []
    for s in range(SAMPLES):
        sampled_grammar = np.random.rand((len(grammar))) < grammar
        strat1_cons = orderConstraints(rankings[:strat_size], sampled_grammar[:strat_size])
        strat2_cons = orderConstraints(rankings[strat_size:], sampled_grammar[strat_size:])
        correct_count = 0
        total_count = 0
        for i, input_i in enumerate(TD_input):
            intermed_form = getWinner(input_i, strat1_cons, ur2cands[input_i])
            surface_form = getWinner(intermed_form, strat2_cons, ur2cands[intermed_form])
            total_count += 1
            if re.sub("_", "", surface_form) == TD_output[i]:
                correct_count += 1
        samp_accs.append(float(correct_count)/float(total_count))
    return np.mean(samp_accs)

def accByDatum (grammar):
    samp_accs = {d:[] for d in TD_input}
    for s in range(SAMPLES):
        sampled_grammar = np.random.rand((len(grammar))) < grammar
        strat1_cons = orderConstraints(rankings[:strat_size], sampled_grammar[:strat_size])
        strat2_cons = orderConstraints(rankings[strat_size:], sampled_grammar[strat_size:])
        for i, input_i in enumerate(TD_input):
            intermed_form = getWinner(input_i, strat1_cons, ur2cands[input_i])
            surface_form = getWinner(intermed_form, strat2_cons, ur2cands[intermed_form])
            if re.sub("_", "", surface_form) == TD_output[i]:
                samp_accs[input_i].append(1.0)
            else:
                samp_accs[input_i].append(0.0)
    av_accs = [np.mean(samp_accs[ur]) for ur in TD_input]
    return av_accs   
    
def forcedChoiceByDatum (grammar):
    lang2ur2choice =  { 
                    'esi_B':{
                                "Ese":     ["Ese", "ESe"],
                                "esE":     ["esE", "eSE"],
                                "ake":     ["ake", "aki"],
                                "akE":     ["akE", "akI"],
                                "aki":     ["aki", "ake"],
                                "akI":     ["akI", "akE"],
                                "ase":     ["ase", "aSe"],
                                "asE":     ["asE", "aSE"],
                                "ekI":     ["ekE", "ekI"],
                                "Eki":     ["Eke", "Eki"],
                                "ikE":     ["ikI", "ikE"],
                                "Ike":     ["Iki", "Ike"],
                                "isI":     ["iSI", "isI"],
                                "Isi":     ["ISi", "Isi"],
                                "asi":     ["aSi", "asi"],
                                "asI":     ["aSI", "asI"],
                                "esi":     ["ese", "eSe"],
                                "Esi":     ["Ese", "ESe"],
                                "esI":     ["esE", "eSE"],
                                "EsI":     ["EsE", "ESE"],
                            },
                    'ise_F':{
                                "Ese":     ["Ese", "ESe"],
                                "esE":     ["esE", "eSE"],
                                "ake":     ["ake", "aki"],
                                "akE":     ["akE", "akI"],
                                "aki":     ["aki", "ake"],
                                "akI":     ["akI", "akE"],
                                "ase":     ["ase", "aSe"],
                                "asE":     ["asE", "aSE"],
                                "ekI":     ["ekE", "ekI"],
                                "Eki":     ["Eke", "Eki"],
                                "ikE":     ["ikI", "ikE"],
                                "Ike":     ["Iki", "Ike"],
                                "isI":     ["iSI", "isI"],
                                "Isi":     ["ISi", "Isi"],
                                "asi":     ["aSi", "asi"],
                                "asI":     ["aSI", "asI"],
                                "ise":     ["iSi", "isi"],
                                "Ise":     ["ISi", "Isi"],
                                "isE":     ["iSI", "isI"],
                                "IsE":     ["ISI", "IsI"],
                            },
                    'esi_CB':{
                                "Ese":     ["Ese", "ESe"],
                                "esE":     ["esE", "eSE"],
                                "ake":     ["ake", "aki"],
                                "akE":     ["akE", "akI"],
                                "aki":     ["aki", "ake"],
                                "akI":     ["akI", "akE"],
                                "ase":     ["ase", "aSe"],
                                "asE":     ["asE", "aSE"],
                                "ekI":     ["ekE", "ekI"],
                                "Eki":     ["Eke", "Eki"],
                                "ikE":     ["ikI", "ikE"],
                                "Ike":     ["Iki", "Ike"],
                                "isI":     ["iSI", "isI"],
                                "Isi":     ["ISi", "Isi"],
                                "asi":     ["aSi", "asi"],
                                "asI":     ["aSI", "asI"],
                                "esi":     ["eSe", "ese"],
                                "Esi":     ["ESe", "Ese"],
                                "esI":     ["eSE", "esE"],
                                "EsI":     ["ESE", "EsE"],
                            },
                    'ise_CF':{
                                "Ese":     ["Ese", "ESe"],
                                "esE":     ["esE", "eSE"],
                                "ake":     ["ake", "aki"],
                                "akE":     ["akE", "akI"],
                                "aki":     ["aki", "ake"],
                                "akI":     ["akI", "akE"],
                                "ase":     ["ase", "aSe"],
                                "asE":     ["asE", "aSE"],
                                "ekI":     ["ekE", "ekI"],
                                "Eki":     ["Eke", "Eki"],
                                "ikE":     ["ikI", "ikE"],
                                "Ike":     ["Iki", "Ike"],
                                "isI":     ["iSI", "isI"],
                                "Isi":     ["ISi", "Isi"],
                                "asi":     ["aSi", "asi"],
                                "asI":     ["aSI", "asI"],
                                "ise":     ["isi", "iSi"],
                                "Ise":     ["Isi", "ISi"],
                                "isE":     ["isI", "iSI"],
                                "IsE":     ["IsI", "ISI"],
                            }
                  }
    samp_accs = {d:[] for d in TD_input}
    for s in range(SAMPLES):
        sampled_grammar = np.random.rand((len(grammar))) < grammar
        strat1_cons = orderConstraints(rankings[:strat_size], sampled_grammar[:strat_size])
        strat2_cons = orderConstraints(rankings[strat_size:], sampled_grammar[strat_size:])
        for i, input_i in enumerate(TD_input):
            intermed_form = getWinner(input_i, strat1_cons, ur2cands[input_i])
            surface_form = getWinner(intermed_form, strat2_cons, ur2cands[intermed_form])
            if re.sub("_", "", surface_form) == lang2ur2choice[LANGUAGE][input_i][0]:
                samp_accs[input_i].append(1.0) #Correct choice
            elif re.sub("_", "", surface_form) == lang2ur2choice[LANGUAGE][input_i][1]:
                samp_accs[input_i].append(0.0) #Incorrect choice
    av_accs = [np.mean(samp_accs[ur]) for ur in TD_input]
    return av_accs   
    
def sampleTestDatum (grammar):
    sample_counts = {}
    for s in range(SAMPLES):
        sampled_grammar = np.random.rand((len(grammar))) < grammar
        strat1_cons = orderConstraints(rankings[:strat_size], sampled_grammar[:strat_size])
        strat2_cons = orderConstraints(rankings[strat_size:], sampled_grammar[strat_size:])
        intermed_form = getWinner(TEST_DATUM, strat1_cons, ur2cands[TEST_DATUM])
        surface_form = getWinner(intermed_form, strat2_cons, ur2cands[intermed_form])
        try:
            sample_counts[surface_form] += 1.0
        except:
            sample_counts[surface_form] = 1.0
    
    return sample_counts  



## Run Simulations

In [0]:
###START SIMULATIONS###
solution_counts = {}
learning_curves = []
datum_curves = {ur:[] for ur in TD_input}
datum_fc_curves = {ur:[] for ur in TD_input}
test_probs = {} 
gen_curves = {v:[[] for r in range(REPS)] for v in TEST_DATUM_OUTPUTS.values()}  
all_rank_probs = {r+" (Stratum:"+str(st+1)+")":[] for r in rankings for st in range(STRAT_NUM)}
for rep in range(REPS):                
    #Initialize the grammar:
    rank_probs = np.array([.5 for rank in rankings])
    
    #Start learning:
    con_ranks_s1 = {c:[] for c in CON}
    con_ranks_s2 = {c:[] for c in CON}
    this_curve = []
    this_datum_curve = {ur:[] for ur in TD_input}
    this_datum_fc_curve = {ur:[] for ur in TD_input}
    for epoch in range(EPOCHS):
        if epoch % 25 == 0:
          print ("Epoch: ", epoch, "Rep: ", rep)
        av_ranks1 = {}
        av_ranks2 = {}
        for TD_index, this_UR in enumerate(TD_input):
            new_grammar = [0 for r in rankings]
            for rank_i, ranking in enumerate(rankings):
                freq_given_true = 0
                freq_given_false = 0
                for sample in range(SAMPLES):
                    #Sample grammar:
                    sampled_grammar = np.random.rand((len(rankings))) < rank_probs
                    
                    #Set the ranking of interest to true/false:
                    true_grammar = np.copy(sampled_grammar)
                    true_grammar[rank_i] = True
                    false_grammar = np.copy(sampled_grammar)
                    false_grammar[rank_i] = False              
                    
                    #Get the ordered list of constraints:
                    true_dom_counts_s1 = {c:0 for c in CON}
                    false_dom_counts_s1 = {c:0 for c in CON}
                    true_dom_counts_s2 = {c:0 for c in CON}
                    false_dom_counts_s2 = {c:0 for c in CON}
                    for rank_j, rank_name in enumerate(rankings):
                        if rank_j < strat_size:
                        #Stratum 1:
                            #True:
                            if true_grammar[rank_j]:
                                true_dom_counts_s1[rank_name.split(">>")[0]] += 1
                            else:
                                true_dom_counts_s1[rank_name.split(">>")[1]] += 1
                            #False:
                            if false_grammar[rank_j]:
                                false_dom_counts_s1[rank_name.split(">>")[0]] += 1
                            else:
                                false_dom_counts_s1[rank_name.split(">>")[1]] += 1
                        else:
                        #Stratum 2:
                            #True:
                            if true_grammar[rank_j]:
                                true_dom_counts_s2[rank_name.split(">>")[0]] += 1
                            else:
                                true_dom_counts_s2[rank_name.split(">>")[1]] += 1
                            #False:
                            if false_grammar[rank_j]:
                                false_dom_counts_s2[rank_name.split(">>")[0]] += 1
                            else:
                                false_dom_counts_s2[rank_name.split(">>")[1]] += 1    
                                
                    #Stratum 1
                    true_ranked_s1 = []
                    false_ranked_s1 = []
                    for constraint in sorted(true_dom_counts_s1.items(), key=operator.itemgetter(1), reverse=True):
                        true_ranked_s1.append(constraint[0])
                        #con_ranks_s1[constraint[0]].append(true_dom_counts_s1[constraint[0]])
                    for constraint in sorted(false_dom_counts_s1.items(), key=operator.itemgetter(1), reverse=True):
                        false_ranked_s1.append(constraint[0])
                        #con_ranks_s1[constraint[0]].append(false_dom_counts_s1[constraint[0]])
                    #Stratum 2
                    true_ranked_s2 = []
                    false_ranked_s2 = []
                    for constraint in sorted(true_dom_counts_s2.items(), key=operator.itemgetter(1), reverse=True):
                        true_ranked_s2.append(constraint[0])
                        #con_ranks_s2[constraint[0]].append(true_dom_counts_s2[constraint[0]])
                    for constraint in sorted(false_dom_counts_s2.items(), key=operator.itemgetter(1), reverse=True):
                        false_ranked_s2.append(constraint[0])
                        #con_ranks_s2[constraint[0]].append(false_dom_counts_s2[constraint[0]])
                        
                    #Get predicted winners, check them:
                    #Stratum 1
                    true_s1_winner = getWinner(this_UR, true_ranked_s1, ur2cands[this_UR])
                    false_s1_winner = getWinner(this_UR, false_ranked_s1, ur2cands[this_UR])
                    #Stratum 2
                    true_s2_winner = getWinner(true_s1_winner, true_ranked_s2, ur2cands[true_s1_winner])
                    false_s2_winner = getWinner(false_s1_winner, false_ranked_s2, ur2cands[false_s1_winner])
                    
                    #Record learning curve info:
                    for c in CON:
                        try:
                            av_ranks1[c].append(true_ranked_s1.index(c))
                            av_ranks2[c].append(true_ranked_s2.index(c))
                        except:
                            av_ranks1[c] = [true_ranked_s1.index(c)]
                            av_ranks2[c] = [true_ranked_s2.index(c)]
                    
                    if re.sub("_", "", true_s2_winner) == TD_output[TD_index]:
                        freq_given_true += 1.0
                    if re.sub("_", "", false_s2_winner) == TD_output[TD_index]:
                        freq_given_false += 1.0
                
                #Use the sample drawn above to calculate the necessary probs:    
                prob_given_true = (freq_given_true/SAMPLES) + .0001
                prob_given_false = (freq_given_false/SAMPLES) + .0001
                probDatumGivenGrammar = (prob_given_true * rank_probs[rank_i]) + \
                                        (prob_given_false * (1.0 - rank_probs[rank_i]))\
                                        + .0001
                new_grammar[rank_i] = (prob_given_true * rank_probs[rank_i])/probDatumGivenGrammar
            
            rank_probs = (np.array(new_grammar)*LEARNING_RATE) + (rank_probs*(1-LEARNING_RATE))
            
        #Store learning curve information:
        for c in CON:
            con_ranks_s1[c].append(-1*np.mean(av_ranks1[c]))
            con_ranks_s2[c].append(-1*np.mean(av_ranks2[c]))
        this_curve.append(getAccuracy(rank_probs))
        these_accs = accByDatum(rank_probs)
        these_fc_accs = forcedChoiceByDatum(rank_probs)
        for i, input_i in enumerate(TD_input):
            this_datum_curve[input_i].append(these_accs[i])
            this_datum_fc_curve[input_i].append(these_fc_accs[i])

        #Store generalization curve information:
        testCounts = sampleTestDatum(rank_probs)
        totalCount = float(sum(testCounts.values()))
        for test_type in TEST_DATUM_OUTPUTS.keys():
          sr = TEST_DATUM_OUTPUTS[test_type]
          if sr in testCounts.keys():
            try:
              gen_curves[sr][rep].append(testCounts[sr]/totalCount)
            except:
              print ("\n\n!!\n", sr, rep, "\n\n!!\n")
              raise Exception
          else:
            try:
              gen_curves[sr][rep].append(0.0)
            except:
              print ("\n\n!!\n", gen_curves.keys())
              print ("\n\n!!\n", sr, rep, "\n\n!!\n")
              raise Exception

    #Track solutions that were found:
    final_sample = np.random.rand((len(rankings))) < rank_probs
    final_strat1 = orderConstraints(rankings[:strat_size], final_sample[:strat_size])
    final_strat2 = orderConstraints(rankings[strat_size:], final_sample[strat_size:])
    solution = "Strat1: "+">>".join(final_strat1)+", Strat2: "+">>".join(final_strat2) 
    try:
        solution_counts[solution][0] += 1
        solution_counts[solution][1].append(this_curve[-1])
    except:
        solution_counts[solution] = [1, [this_curve[-1]]] 
        
    #Track ranking probs that were found:
    ranks_so_far = []
    strat = 1
    for rank_i, rank_name in enumerate(rankings):
        if rank_name in ranks_so_far:
            strat = 2
        ranks_so_far.append(rank_name)
        all_rank_probs[rank_name+" (Stratum:"+str(strat)+")"].append(rank_probs[rank_i])   
        
    #Track learning:
    learning_curves.append(this_curve)
    for i, input_i in enumerate(TD_input):
            datum_curves[input_i].append(this_datum_curve[input_i])
            datum_fc_curves[input_i].append(this_datum_fc_curve[input_i])
    
    #Track generalization to withheld data:
    testCounts = sampleTestDatum(rank_probs)
    totalCount = float(sum(testCounts.values()))
    for sampled_output in testCounts.keys():
        try:
            test_probs[sampled_output].append(testCounts[sampled_output]/totalCount)
        except:
            test_probs[sampled_output] = [testCounts[sampled_output]/totalCount]

## Display/Save Results

In [0]:
###SAVE/PLOT RESULTS###
#Save test data output:
testData_file = open("/content/gdrive/My Drive/"+WD+"/Output Files/"+LANGUAGE+"_testData (reps="+str(REPS)+").csv", "w")
for output in test_probs.keys():
    full_list = test_probs[output]+[0.0]*(REPS-len(test_probs[output])) 
    testData_file.write(output+","+str(np.mean(full_list))+","+str(sem(full_list))+"\n")  
testData_file.close()         

#Save overall learning curve:
curve_file = open("/content/gdrive/My Drive/"+WD+"/Output Files/"+LANGUAGE+"_curveOutput (reps="+str(REPS)+").txt", "w")
av_curve = np.mean(learning_curves, axis=0)
for epoch in av_curve:
    curve_file.write(str(epoch)+"\n")
curve_file.close()

#Save solution counts:
solution_file = open("/content/gdrive/My Drive/"+WD+"/Output Files/"+LANGUAGE+"_solutionCountOutput (reps="+str(REPS)+").txt", "w")
for sol in solution_counts.keys():
    solution_file.write(sol+"\t"+str(solution_counts[sol][0])+"\t"+str(np.mean(solution_counts[sol][1]))+"\n")
solution_file.close()

#Save ranking prob distributions:
rankProb_file = open("/content/gdrive/My Drive/"+WD+"/Output Files/"+LANGUAGE+"_rankingProbs (reps="+str(REPS)+").csv", "w")
for strat in range(1,STRAT_NUM+1):
    rankProb_file.write("STRATUM: "+str(strat)+"\n")
    rankProb_file.write(","+",".join(CON)+"\n")
    for row in CON:
        rankProb_file.write(row+",")
        for column in CON:
            if column == row:
                rankProb_file.write("-,")
                continue
            try:
                these_probs = all_rank_probs[row+">>"+column+" (Stratum:"+str(strat)+")"]
            except:
                these_probs = [1-rp for rp in all_rank_probs[column+">>"+row+" (Stratum:"+str(strat)+")"]]
            average = np.around(np.mean(these_probs), 3)
            sd = np.around(np.std(these_probs), 3)
            rankProb_file.write(str(average)+" ("+str(sd)+"),")
        rankProb_file.write("\n")               
rankProb_file.close()

#Save generalization curves:
genCurve_file = open("/content/gdrive/My Drive/"+WD+"/Output Files/"+LANGUAGE+"_GenCurves (reps="+str(REPS)+").csv", "w")
for sr in gen_curves.keys():
    for rep_ix, rep in enumerate(gen_curves[sr]): 
      genCurve_file.write(sr+","+"Rep:"+str(rep_ix)+","+",".join([str(p) for p in rep])+"\n")  
genCurve_file.close() 


#Plot datum (and forced choice) accuracy curves: 
byDatum_file = open("/content/gdrive/My Drive/"+WD+"/Output Files/"+LANGUAGE+"_byDatumOutput (reps="+str(REPS)+").csv", "w")   
forcedChoice_file =   open("/content/gdrive/My Drive/"+WD+"/Output Files/"+LANGUAGE+"_forcedChoiceOutput (reps="+str(REPS)+").csv", "w")             
for datum in datum_curves.keys():
    av_datum_curve = np.mean(datum_curves[datum], axis=0)
    #av_fc_curve = np.mean(datum_fc_curves[datum], axis=0)
    byDatum_file.write(datum+","+",".join([str(a) for a in av_datum_curve])+"\n")
    #forcedChoice_file.write(datum+","+",".join([str(a) for a in av_fc_curve])+"\n")

    for rep, curve in enumerate(datum_fc_curves[datum]):
      this_row = [str(rep), str(datum)]
      for epoch, prob in enumerate(curve):
        this_row.append(str(prob))
      forcedChoice_file.write(",".join(this_row)+"\n")


    plot(av_datum_curve, label=datum, linestyle="-")
byDatum_file.close()
forcedChoice_file.close()

l = legend()
xlabel("Epoch")
ylabel("Average Percent Correct")
title(LANGUAGE)
show() 